In [ ]:
import pandas as pd
import re

police_path = "울릉경찰서 교통계(교통사고)_2025년도.csv"
ulleung_path = "ulleung_accidents_with_coords.csv"

police = pd.read_csv(police_path)
ulleung = pd.read_csv(ulleung_path)

place_col = [c for c in police.columns if "사고" in c and "장소" in c][0]
type_col = [c for c in police.columns if "종별" in c][0]


def normalize(text):
    if pd.isna(text):
        return ""
    text = str(text).replace("
", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

police = police[[place_col, type_col]].copy()
police[place_col] = police[place_col].map(normalize)
police[type_col] = police[type_col].map(normalize)
police = police[(police[place_col] != "") & (police[type_col] != "")]

police = police.drop_duplicates(subset=[place_col])

mapping = dict(zip(police[place_col], police[type_col]))

ulleung["type"] = ulleung["raw"].map(lambda s: mapping.get(normalize(s), pd.NA))
ulleung.to_csv(ulleung_path, index=False)

missing = ulleung[ulleung["type"].isna()][["raw"]]
print("missing count", len(missing))
print(missing.to_string(index=False))


In [13]:
import pandas as pd
import re
import unicodedata
from pathlib import Path

# 파일 찾기
def find_target_files(root: Path):
    # _with_coords.csv 로 끝나는 파일들을 찾습니다.
    return sorted(list(root.glob("*_with_coords.csv")))

def normalize_text(text):
    if pd.isna(text):
        return ""
    # 유니코드 정규화 및 줄바꿈/다중공백 제거
    text = unicodedata.normalize("NFC", str(text))
    text = text.replace("\n", " ").strip()
    return re.sub(r'\s+', ' ', text)

def parse_address(full_addr):
    """
    주소 문자열을 분석하여 (clean_addr, detail) 튜플을 반환합니다.
    예: "경북 울릉군 ...로 550 경북 울릉군 ...로 550 식당앞" 
    -> ("경상북도 울릉군 ...로 550", "식당앞")
    """
    full_addr = normalize_text(full_addr)
    if not full_addr:
        return "", ""

    # 1. 정규식 정의
    # 도로명 주소 패턴: 시/도 + 시/군/구 + 로/길 + 번호
    # (공백이 불규칙할 수 있으므로 \s* 사용, 번호는 123-4 형식 포함)
    pat_road = r'((?:경상북도|경북)\s+(?:울릉군|울릉)\s+[가-힣0-9]+(?:로|길)\s+\d+(?:\s*-\s*\d+)?)'
    
    # 지번 주소 패턴: 시/도 + 시/군/구 + 읍/면 + (리) + 번지
    # 예: 경북 울릉 북면 현포 296- 2번지
    pat_jibun = r'((?:경상북도|경북)\s+(?:울릉군|울릉)\s+[가-힣]+(?:읍|면|동)(?:\s+[가-힣]+리)?\s+\d+(?:\s*-\s*\d+)?(?:번지)?)'

    # 매칭 시도 (도로명 우선)
    match = re.search(pat_road, full_addr)
    if not match:
        match = re.search(pat_jibun, full_addr)
    
    if match:
        clean_addr = match.group(1)
        # 정규화: '경북' -> '경상북도', '울릉' -> '울릉군' 등 표준화가 필요하면 여기서 처리
        # 일단은 추출된 문자열을 공백 정리하여 사용
        clean_addr = re.sub(r'\s+', ' ', clean_addr).strip()
        
        # detail 추출 로직
        # 1. 원본 문자열에서 clean_addr 부분을 제거해본다.
        # 단순히 replace를 쓰면 중간에 있는 것도 지워질 수 있으므로, 
        # "주소가 문자열의 시작 부분에 반복해서 나오는지" 확인하는 것이 좋습니다.
        
        # 정규식 매치된 부분의 끝 인덱스
        end_pos = match.end()
        remainder = full_addr[end_pos:].strip()
        
        # 만약 남은 문자열이 또다시 주소로 시작한다면? (중복 기재된 경우)
        # 예: "주소 주소 상세" -> " 상세"
        # 정확한 문자열 매칭보다는, 정규식으로 다시 확인
        match_again = re.match(pat_road, remainder) or re.match(pat_jibun, remainder)
        if match_again:
            # 두 번째 주소 뒤에 있는 부분이 진짜 detail
            detail = remainder[match_again.end():].strip()
        else:
            # 주소가 한 번만 나온 경우, 그 뒤가 detail
            detail = remainder
            
        return clean_addr, detail
    else:
        # 주소 패턴을 못 찾은 경우
        return "", full_addr

def process_file(path: Path):
    print(f"Processing: {path.name}")
    
    # CSV 읽기
    try:
        df = pd.read_csv(path, encoding='utf-8-sig')
    except:
        df = pd.read_csv(path, encoding='cp949') # fallback

    # 컬럼 이름 공백 정리
    df.columns = [c.strip() for c in df.columns]

    # 원본 주소 컬럼 찾기 (raw가 있으면 raw, 없으면 사고장소)
    src_col = 'raw' if 'raw' in df.columns else '사고장소'
    if src_col not in df.columns:
        # 둘 다 없으면 가장 긴 문자열을 가진 컬럼을 주소로 추정하거나 스킵
        print(f"  Skipping: source address column not found.")
        return

    # 새로운 데이터 리스트 생성
    clean_list = []
    detail_list = []
    
    for val in df[src_col]:
        c, d = parse_address(val)
        clean_list.append(c)
        detail_list.append(d)

    # 결과 데이터프레임 생성 (순서 지정)
    # 기존 lat/lon, type이 있다면 유지
    out_df = pd.DataFrame()
    out_df['raw'] = df[src_col].apply(normalize_text) # 원본
    out_df['detail'] = detail_list
    out_df['clean_normalized'] = clean_list
    
    if 'latitude' in df.columns:
        out_df['latitude'] = df['latitude']
    if 'longitude' in df.columns:
        out_df['longitude'] = df['longitude']
        
    # type 컬럼 찾기 (사고종별, type 등)
    type_col = next((c for c in df.columns if '종별' in c or 'type' in c.lower()), None)
    if type_col:
        out_df['type'] = df[type_col]

    # 저장
    out_df.to_csv(path, index=False, encoding='utf-8-sig')
    print(f"  Done. Saved to {path.name}")

# 실행
files = find_target_files(Path("."))
if not files:
    print("No target files found (*_with_coords.csv).")
else:
    for f in files:
        process_file(f)

Processing: ulleung_accidents_with_coords.csv
  Done. Saved to ulleung_accidents_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2019년도_with_coords.csv
  Done. Saved to 울릉경찰서 교통계(교통사고)_2019년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2020년도_with_coords.csv
  Done. Saved to 울릉경찰서 교통계(교통사고)_2020년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2021년도_with_coords.csv
  Done. Saved to 울릉경찰서 교통계(교통사고)_2021년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2022년도_with_coords.csv
  Done. Saved to 울릉경찰서 교통계(교통사고)_2022년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2023년도_with_coords.csv
  Done. Saved to 울릉경찰서 교통계(교통사고)_2023년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2024년도_with_coords.csv
  Done. Saved to 울릉경찰서 교통계(교통사고)_2024년도_with_coords.csv


In [2]:
import csv

sms_path = "울릉알리미_텍스트.csv"

names = set()
with open(sms_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        name = (row.get("sms_mem_name") or "").strip()
        if name:
            names.add(name)

print(sorted(names))


['KT', '감염병예방의약팀', '감영병예방의약팀', '건강출산팀', '건설과', '건영택배', '경제교통정책실', '경제투자유치실', '공보팀', '공항건설행정팀', '관광개발팀', '관광기획팀', '관광문화체육과', '관광문화체육실', '관광산림과', '관제센터', '교육인구정책팀', '교통정책과', '교통지도팀', '교통항공팀', '교통행정팀', '국민건강보험공단', '금광해운', '기술보급과', '기술보급팀', '기획감사실', '기획법무팀', '긴급재난지원금지원팀', '농업기술센터', '농업산림과', '농업유통과', '농정팀', '대저건설', '대저페리', '대저해운', '대저해운 도착시간', '도로토목팀', '도시건축과', '독도관리사무소', '독도박물관', '독도크루즈', '롯데택배', '문화예술팀', '문화체육과', '미래전략추진단', '미래전략팀', '미래해운', '민원팀', '방재하천팀', '보건사업과', '보건의료원', '북면사무소', '산림팀', '상하수도사업소', '서면사무소', '세정팀', '수산정책팀', '시설관리사업소', '썬라이즈 도착시간', '씨스포빌', '씨스포빌 도착시간', '안전건설과', '안전건설단', '안전도시과', '안전정책팀', '언론홍보팀', '에이치해운', '여객항만팀', '여성가족팀', '우성해운', '울릉경비대', '울릉경찰서', '울릉경찰서 교통계', '울릉경찰서 범죄예방계', '울릉경찰서 생활안전교통과', '울릉경찰서 수사과', '울릉교육지원청', '울릉군 민방위상황실', '울릉군 재난안전대책본부', '울릉군가족센터', '울릉군긴급재난지원팀', '울릉군여성단체협의회', '울릉군청', '울릉도관측소', '울릉도독도해양연구기지', '울릉문화원', '울릉우체국', '울릉읍사무소', '울릉크루즈', '원무팀', '위생팀', '의회사무과', '인구정책팀', '인재육성팀', '일자리경제교통과', '일자리경제팀', '자치행정과', '재무과', '정보통신팀', '제이에이치페리', '제이에이치페리 도착시간', '주민복지과

In [10]:
import csv

sms_path = "울릉알리미_텍스트.csv"
out_path = "sms_msg_classified.csv"

ship_keywords = [
    "금광해운",
    "대저해운",
    "대저해운 도착시간",
    "에이치해운",
    "우성해운",
    "주식회사태성해운",
    "태성해운 도착시간",
]
people_keywords = [
    "대저페리",
    "썬라이즈 도착시간",
    "씨스포빌",
    "씨스포빌 도착시간",
    "울릉크루즈",
    "제이에이치페리",
    "제이에이치페리 도착시간",
]
cancel_keywords = [
    "결항",
    "운항 통제",
    "운항통제",
    "운항이 통제",
    "운항없음",
    "운항 없음",
    "운항 취소",
    "출항 취소",
    "출항불가",
    "휴항",
    "운항 중지",
    "취소되었습니다",
    "통제되었습니다",
]
depart_keywords = [
    "출항",
    "출발",
    "운항합니다",
    "정상운항",
    "운항 예정",
]
arrive_keywords = [
    "입항 예정",
    "입항 예정시간",
    "입항입니다",
    "입항 예정 시간",
]
exclude_keywords = [
    "셔틀",
]

def classify_category(msg: str) -> str:
    if not msg:
        return "미분류"
    for k in ship_keywords:
        if k in msg:
            return "선박"
    for k in people_keywords:
        if k in msg:
            return "사람"
    return "미분류"

def classify_status(msg: str) -> str:
    if not msg:
        return "미정"
    for k in cancel_keywords:
        if k in msg:
            return "운항불가"
    for k in arrive_keywords:
        if k in msg:
            return "입항 예정"
    for k in depart_keywords:
        if k in msg:
            return "출항"
    return "미정"

with open(sms_path, newline="", encoding="utf-8") as f, open(
    out_path, "w", newline="", encoding="utf-8"
) as out:
    reader = csv.DictReader(f)
    fieldnames = ["sms_msg", "sms_resDate", "category", "status"]
    writer = csv.DictWriter(out, fieldnames=fieldnames)
    writer.writeheader()
    for row in reader:
        msg = (row.get("sms_msg") or "").strip()
        if any(k in msg for k in exclude_keywords):
            continue
        category = classify_category(msg)
        if category == "미분류":
            continue
        status = classify_status(msg)
        writer.writerow({
            "sms_msg": msg,
            "sms_resDate": (row.get("sms_resDate") or "").strip(),
            "category": category,
            "status": status,
        })

print("saved", out_path)


saved sms_msg_classified.csv


In [11]:
import csv
from collections import Counter
from datetime import datetime

classified_path = "sms_msg_classified.csv"

counts = Counter()
with open(classified_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        res_date = (row.get("sms_resDate") or "").strip()
        if not res_date:
            continue
        res_date = res_date.replace(".", "-").replace("/", "-")
        try:
            dt = datetime.strptime(res_date, "%Y-%m-%d %H:%M")
        except ValueError:
            try:
                dt = datetime.strptime(res_date, "%Y-%m-%d")
            except ValueError:
                continue
        if dt.year != 2025:
            continue
        status = (row.get("status") or "미정").strip()
        counts[status] += 1

print("2025 status counts")
for k, v in counts.most_common():
    print(f"{k}: {v}")


2025 status counts
입항 예정: 295
미정: 13


In [12]:
import csv
from datetime import datetime

classified_path = "sms_msg_classified.csv"

def parse_dt(value: str):
    value = value.replace(".", "-").replace("/", "-")
    for fmt in ("%Y-%m-%d %H:%M", "%Y-%m-%d"):
        try:
            return datetime.strptime(value, fmt)
        except ValueError:
            continue
    return None

printed = 0
with open(classified_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        status = (row.get("status") or "").strip().replace(" ", "")
        if status != "입항예정":
            continue
        res_date = (row.get("sms_resDate") or "").strip()
        dt = parse_dt(res_date)
        if not dt or dt.year != 2025:
            continue
        print(row.get("sms_msg"))
        printed += 1
        if printed >= 10:
            break

print("printed", printed)


금일(01일) 울릉크루즈 울릉(사동항) 입항 예정시간은 08:00분이며, 탑승인원은 1,200명 입니다
금일(02일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 383명 입니다
금일(03일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 237명 입니다
금일(04일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 456명 입니다
금일(05일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 202명 입니다
금일(06일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 391명 입니다
금일(11일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 603명 입니다
금일(12일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 311명 입니다
금일(13일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 246명 입니다
금일(14일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:30분이며, 탑승인원은 418명 입니다
printed 10


In [13]:
import csv
from datetime import datetime

classified_path = "sms_msg_classified.csv"

def parse_dt(value: str):
    value = value.replace(".", "-").replace("/", "-")
    for fmt in ("%Y-%m-%d %H:%M", "%Y-%m-%d"):
        try:
            return datetime.strptime(value, fmt)
        except ValueError:
            continue
    return None

def print_sample(target_status: str, limit: int = 10):
    printed = 0
    with open(classified_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            status = (row.get("status") or "").strip().replace(" ", "")
            if status != target_status:
                continue
            res_date = (row.get("sms_resDate") or "").strip()
            dt = parse_dt(res_date)
            if not dt or dt.year != 2025:
                continue
            print(row.get("sms_msg"))
            printed += 1
            if printed >= limit:
                break
    print("printed", printed)

print("입항예정 샘플")
print_sample("입항예정", 10)

print("")
print("미정 샘플")
print_sample("미정", 10)


입항예정 샘플
금일(01일) 울릉크루즈 울릉(사동항) 입항 예정시간은 08:00분이며, 탑승인원은 1,200명 입니다
금일(02일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 383명 입니다
금일(03일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 237명 입니다
금일(04일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 456명 입니다
금일(05일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 202명 입니다
금일(06일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 391명 입니다
금일(11일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 603명 입니다
금일(12일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 311명 입니다
금일(13일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:20분이며, 탑승인원은 246명 입니다
금일(14일) 울릉크루즈 울릉(사동항) 입항 예정시간은 07:30분이며, 탑승인원은 418명 입니다
printed 10

미정 샘플
엘도라도EX호 2025년 울릉주민 정기권 발행을 시작하였습니다. 자세한 사항은 대저페리 홈페이지를 확인하여 주시기 바랍니다.
*정정*엘도라도EX호 2025년 울릉주민 정기권 3월부터 발행 예정입니다. 자세한 사항은 대저페리 홈페이지를 확인하여 주시기 바랍니다.
내일(28일)부터 엘도라도EX호 2025년 울릉주민 정기권 발행을 시작합니다. 자세한 사항은 대저페리 홈페이지를 확인하여 주시기 바랍니다.
[한진택배] 사동항 울릉크루즈 앞 오전7시부터 당일택배 접수받습니다. 4월1일 부터 5월말까지 주민분들의 많은 관심 부탁드립니다.
? 울릉크루즈 당일 주민 승선권 발권 알림    - 기      간 : 2025. 5. 19.(월)부터    - 내      용 : 군민 당일 승선권 30매    - 발권방법 : 선착순   "울릉군은 군민 

In [4]:
import argparse
import json
import os
import time
import unicodedata
import re
import requests
import pandas as pd
from pathlib import Path

# 설정
TOKEN_TRAFFIC_STATS = "교통계"
TOKEN_ACCIDENT = "교통사고"
TOKEN_YEAR = "년도"
ADDRESS_COL = "사고장소"
LAT_COL = "latitude"
LON_COL = "longitude"

def _normalize_name(name: str) -> str:
    return unicodedata.normalize("NFC", name)

def _clean_address(addr):
    """
    주소 문자열을 정제합니다. 
    예: '경북 울릉군 ...로 550 경북 울릉군 ...로 550 식당앞' -> '경상북도 울릉군 울릉순환로 550'
    """
    if not isinstance(addr, str):
        return ""
    
    # 1. 도로명 주소 추출 (시/도 + 시/군/구 + 로/길 + 번호)
    # 정규식: (경북|경상북도) (울릉군|...) (무슨로|무슨길) (숫자)
    pattern_road = r'(([가-힣]+도|[가-힣]+시)\s+[가-힣]+(시|군|구)\s+[가-힣0-9]+(로|길)\s+\d+(-\d+)?)'
    match = re.search(pattern_road, addr)
    if match:
        return match.group(1)
    
    # 2. 지번 주소 추출 (시/도 + 시/군/구 + 읍/면/동 + 번호)
    pattern_jibun = r'(([가-힣]+도|[가-힣]+시)\s+[가-힣]+(시|군|구)\s+[가-힣]+(읍|면|동)\s+[가-힣0-9]+\s+\d+(-\d+)?)'
    match = re.search(pattern_jibun, addr)
    if match:
        return match.group(1)
        
    # 3. 추출 실패 시, 중복이라도 제거 시도 (공백 기준 split 후 중복 제거)
    tokens = addr.split()
    seen = set()
    result = []
    for token in tokens:
        if token not in seen:
            seen.add(token)
            result.append(token)
    
    return " ".join(result)

def geocode_nominatim(address: str) -> tuple[float | None, float | None]:
    """Nominatim(OSM)을 사용하여 무료로 좌표를 조회합니다."""
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": address, "format": "json", "limit": 1}
    # Nominatim은 User-Agent가 필수입니다.
    headers = {"User-Agent": "traffic-analysis-script/1.0"}
    
    try:
        resp = requests.get(url, params=params, headers=headers, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        if not data:
            return None, None
        return float(data[0]["lat"]), float(data[0]["lon"])
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

def process_file(path: Path):
    print(f"Processing: {path.name}")
    
    # 여러 인코딩 시도
    df = None
    for enc in ("utf-8-sig", "utf-8", "cp949", "euc-kr"):
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except Exception:
            continue
            
    if df is None:
        print(f"Failed to read {path.name}")
        return

    # 컬럼명 공백 제거
    df.columns = [str(c).strip() for c in df.columns]
    
    if ADDRESS_COL not in df.columns:
        print(f"Skipping: '{ADDRESS_COL}' column not found.")
        return

    # 주소 정제
    df["_clean_addr"] = df[ADDRESS_COL].apply(_clean_address)
    unique_addrs = df["_clean_addr"].unique()
    print(f"Unique addresses to geocode: {len(unique_addrs)}")

    # 좌표 캐시 (메모리)
    addr_cache = {}

    from tqdm import tqdm # 진행률 표시 (없으면 제거 가능)
    
    # 지오코딩 수행
    for addr in tqdm(unique_addrs, desc="Geocoding"):
        if not addr:
            continue
        if addr in addr_cache:
            continue
            
        lat, lon = geocode_nominatim(addr)
        if lat and lon:
            addr_cache[addr] = (lat, lon)
        else:
            # 실패 시 원본 주소로 한 번 더 시도해볼 수도 있음
            pass
            
        # Nominatim 정책 준수 (초당 1회 제한)
        time.sleep(1.0)

    # 데이터프레임에 매핑
    df[LAT_COL] = df["_clean_addr"].map(lambda x: addr_cache.get(x, (None, None))[0])
    df[LON_COL] = df["_clean_addr"].map(lambda x: addr_cache.get(x, (None, None))[1])
    
    # 불필요한 임시 컬럼 삭제
    df = df.drop(columns=["_clean_addr"])

    # 저장
    out_path = path.with_name(f"{path.stem}_with_coords.csv")
    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"Saved to: {out_path}")

def main():
    # 현재 디렉토리에서 대상 파일 찾기
    root = Path(".")
    targets = []
    for f in root.iterdir():
        if not f.is_file(): continue
        name = _normalize_name(f.name)
        if not name.endswith(".csv"): continue
        # 파일명 조건 확인
        if TOKEN_TRAFFIC_STATS in name and TOKEN_ACCIDENT in name:
            targets.append(f)
            
    if not targets:
        print("No target CSV files found.")
        # 만약 파일을 못 찾으면 모든 csv를 대상으로 하려면 아래 주석 해제
        # targets = list(root.glob("*.csv"))
    
    for target in targets:
        process_file(target)

if __name__ == "__main__":
    main()

Processing: 울릉경찰서 교통계(교통사고)_2022년도.csv
Unique addresses to geocode: 82


Geocoding: 100%|██████████| 82/82 [02:43<00:00,  1.99s/it]


Saved to: 울릉경찰서 교통계(교통사고)_2022년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2023년도.csv
Unique addresses to geocode: 93


Geocoding: 100%|██████████| 93/93 [03:04<00:00,  1.98s/it]


Saved to: 울릉경찰서 교통계(교통사고)_2023년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2025년도.csv
Unique addresses to geocode: 82


...

Saved to: 울릉경찰서 교통계(교통사고)_2021년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2020년도.csv
Unique addresses to geocode: 78


Geocoding: 100%|██████████| 78/78 [02:25<00:00,  1.87s/it]


Saved to: 울릉경찰서 교통계(교통사고)_2020년도_with_coords.csv
Processing: 울릉경찰서 교통계(교통사고)_2019년도.csv
Unique addresses to geocode: 64


Geocoding: 100%|██████████| 64/64 [01:59<00:00,  1.87s/it]

Saved to: 울릉경찰서 교통계(교통사고)_2019년도_with_coords.csv
